<a href="https://colab.research.google.com/github/ChnssA/-Human-Activity-Recognition-Using-Liquid-Neural-Network-LNN-/blob/main/lnn_for_human_activity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Absolutely. This is a great practice for making your code more organized, modular, and reusable.

I will split the single script into 5 separate Python files:

1.  **`config.py`**: Holds all the constants and configuration settings.
2.  **`data_loader.py`**: Contains the functions to load and preprocess the data.
3.  **`models.py`**: Defines the `BaselineLSTM` and `LtcHARModel` classes.
4.  **`utils.py`**: Includes helper functions for training, evaluating, and counting parameters.
5.  **`main.py`**: The main script that you will run. It imports from the other files to build and run the experiment.

### 📂 New Project Structure

Your project folder `LNN_HAR_Project` should now look like this:

```
LNN_HAR_Project/
├── venv/
├── UCI HAR Dataset/
│   ├── ... (all the dataset files)
├── config.py         # <-- NEW
├── data_loader.py    # <-- NEW
├── models.py         # <-- NEW
├── utils.py          # <-- NEW
└── main.py           # <-- NEW (replaces train_lnn.py)
```

-----

### 🐍 Python Code for Each File

Here is the complete code to copy and paste into each new file.

#### 1\. `config.py`

(This file holds all your settings.)

In [ ]:
# --- Configuration ---
DATASET_PATH = 'UCI HAR Dataset'
BATCH_SIZE = 64
EPOCHS = 15
LEARNING_RATE = 0.001
HIDDEN_UNITS = 48
NUM_CLASSES = 6
INPUT_SIZE = 9  # 9 features (3x body_acc, 3x body_gyro, 3x total_acc)

#### 2\. `data_loader.py`

(This file handles all data loading and preprocessing.)

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

# Import settings from config.py
import config

def load_har_data(split='train'):
    """
    Loads the raw 9-channel time-series data from the 'Inertial Signals'
    folders.
    """
    print(f"Loading {split} data...")
    signals_path = os.path.join(config.DATASET_PATH, split, 'Inertial Signals')

    signal_files = [
        'body_acc_x', 'body_acc_y', 'body_acc_z',
        'body_gyro_x', 'body_gyro_y', 'body_gyro_z',
        'total_acc_x', 'total_acc_y', 'total_acc_z'
    ]

    signal_data = []
    for file_name in signal_files:
        full_path = os.path.join(signals_path, f'{file_name}_{split}.txt')
        data = pd.read_csv(full_path, delim_whitespace=True, header=None).values
        signal_data.append(data)

    # Stack along a new dimension (features) -> (samples, timesteps, features)
    X = np.stack(signal_data, axis=-1)

    # Load labels
    label_path = os.path.join(config.DATASET_PATH, split, f'y_{split}.txt')
    y = pd.read_csv(label_path, header=None).values.flatten()

    # Adjust labels to be 0-indexed (original is 1-6)
    y = y - 1

    return X, y

def create_dataloaders(X_train, y_train, X_test, y_test):
    """
    Applies normalization and creates PyTorch DataLoaders.
    """
    # Reshape for normalization: (samples * timesteps, features)
    X_train_2d = X_train.reshape(-1, X_train.shape[-1])
    X_test_2d = X_test.reshape(-1, X_test.shape[-1])

    # Apply Z-score normalization
    scaler = StandardScaler()
    X_train_scaled_2d = scaler.fit_transform(X_train_2d)
    X_test_scaled_2d = scaler.transform(X_test_2d)

    # Reshape back to 3D: (samples, timesteps, features)
    X_train_scaled_3d = X_train_scaled_2d.reshape(X_train.shape)
    X_test_scaled_3d = X_test_scaled_2d.reshape(X_test.shape)

    # Convert to PyTorch Tensors
    X_train_tensor = torch.tensor(X_train_scaled_3d, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test_scaled_3d, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # Create TensorDatasets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False)

    return train_loader, test_loader

ModuleNotFoundError: No module named 'config'

#### 3\. `models.py`

(This file defines your neural network architectures.)

In [ ]:
import torch
import torch.nn as nn
from ncps.torch import LTC

# A. Baseline Model (LSTM)
class BaselineLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BaselineLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # We only need the output of the last time step
        _, (h_n, _) = self.lstm(x)
        # Get the last hidden state
        out = h_n[-1]
        out = self.fc(out)
        return out

# B. LNN Model (LTC Network)
class LtcHARModel(nn.Module):
    def __init__(self, input_size, units, num_classes):
        super(LtcHARModel, self).__init__()
        # Implements the LTC cell as the recurrent layer
        self.ltc_layer = LTC(input_size, units)
        # A dense softmax output layer
        self.fc = nn.Linear(units, num_classes)

    def forward(self, x):
        # We only need the last hidden state for classification
        _, h_n = self.ltc_layer(x)
        out = self.fc(h_n)
        return out

#### 4\. `utils.py`

(This file contains the reusable helper functions.)

In [ ]:
import torch
from sklearn.metrics import accuracy_score, f1_score
import config

def count_parameters(model):
    """Helper function to count trainable parameters."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def train_model(model, train_loader, criterion, optimizer, epoch):
    """Performs one training epoch."""
    model.train()
    total_loss = 0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{config.EPOCHS}], Train Loss: {avg_loss:.4f}')

def evaluate_model(model, test_loader):
    """Evaluates the model on the test set."""
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro') # Macro F1-score

    return accuracy, f1

#### 5\. `main.py`

(This is the main script you will run. It imports from all the files above.)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Import our custom modules
import config
from data_loader import load_har_data, create_dataloaders
from models import BaselineLSTM, LtcHARModel
from utils import train_model, evaluate_model, count_parameters

def run_experiment():
    # Load data
    X_train, y_train = load_har_data('train')
    X_test, y_test = load_har_data('test')

    # Create DataLoaders
    train_loader, test_loader = create_dataloaders(X_train, y_train, X_test, y_test)

    print("\n--- Model Comparison ---")

    # --- Baseline LSTM ---
    print("\nTraining Baseline LSTM Model...")
    lstm_model = BaselineLSTM(
        input_size=config.INPUT_SIZE,
        hidden_size=config.HIDDEN_UNITS,
        num_layers=2,
        num_classes=config.NUM_CLASSES
    )
    criterion_lstm = nn.CrossEntropyLoss()
    optimizer_lstm = optim.Adam(lstm_model.parameters(), lr=config.LEARNING_RATE)

    for epoch in range(config.EPOCHS):
        train_model(lstm_model, train_loader, criterion_lstm, optimizer_lstm, epoch)

    lstm_accuracy, lstm_f1 = evaluate_model(lstm_model, test_loader)
    lstm_params = count_parameters(lstm_model)

    # --- LNN (LTC) Model ---
    print("\nTraining LNN (LTC) Model...")
    ltc_model = LtcHARModel(
        input_size=config.INPUT_SIZE,
        units=config.HIDDEN_UNITS,
        num_classes=config.NUM_CLASSES
    )
    criterion_ltc = nn.CrossEntropyLoss()
    optimizer_ltc = optim.Adam(ltc_model.parameters(), lr=config.LEARNING_RATE)

    for epoch in range(config.EPOCHS):
        train_model(ltc_model, train_loader, criterion_ltc, optimizer_ltc, epoch)

    ltc_accuracy, ltc_f1 = evaluate_model(ltc_model, test_loader)
    ltc_params = count_parameters(ltc_model)

    # --- Final Results ---
    print("\n\n--- 🏁 Final Results ---")

    print("\nBaseline (LSTM):")
    print(f"  Test Accuracy: {lstm_accuracy * 100:.2f}%")
    print(f"  Macro F1-Score: {lstm_f1:.4f}")
    print(f"  Parameter Count: {lstm_params}")

    print("\nLNN (LTC):")
    print(f"  Test Accuracy: {ltc_accuracy * 100:.2f}%")
    print(f"  Macro F1-Score: {ltc_f1:.4f}")
    print(f"  Parameter Count: {ltc_params}")

    print("\n--- Experiment Objectives Check ---")
    print(f"LTC Accuracy >= 90%? {'Yes' if ltc_accuracy >= 0.9 else 'No'}")

    param_reduction = (lstm_params - ltc_params) / lstm_params
    print(f"Parameter Reduction: {param_reduction * 100:.2f}%")
    print(f"Reduction >= 50%? {'Yes' if param_reduction >= 0.5 else 'No'}")

if __name__ == "__main__":
    run_experiment()

-----

### How to Run

After creating all 5 files, your VS Code setup should be complete. To run the entire project, you just need to run the `main.py` script from your activated terminal:

```bash
python main.py
```

The output will be identical to the previous "big script," but your code is now much cleaner and easier to manage.